In [ ]:
# @title 🚀 ReconCraft — Colab Setup (by brinhosa)
# @markdown Este notebook instala o stack principal de recon, crawling e análise.

!apt update -qq && apt install -yqq golang jq python3-pip git make unzip
!mkdir -p ~/go/bin
!export PATH=$PATH:~/go/bin

# Ferramentas principais ProjectDiscovery
!go install -v github.com/projectdiscovery/httpx/cmd/httpx@latest
!go install -v github.com/projectdiscovery/subfinder/v2/cmd/subfinder@latest
!go install -v github.com/projectdiscovery/naabu/v2/cmd/naabu@latest
!go install -v github.com/projectdiscovery/nuclei/v3/cmd/nuclei@latest
!go install -v github.com/projectdiscovery/katana/cmd/katana@latest
!go install -v github.com/projectdiscovery/dnsx/cmd/dnsx@latest
!go install -v github.com/ProjectAnte/dnsgen@latest

# Ferramentas TomNomNom e afins
!go install -v github.com/tomnomnom/assetfinder@latest
!go install -v github.com/tomnomnom/anew@latest
!go install -v github.com/tomnomnom/waybackurls@latest
!go install -v github.com/lc/gau/v2/cmd/gau@latest

# Outros
!go install -v github.com/jaeles-project/gospider@latest
!go install -v github.com/hakluke/hakrawler@latest
!go install -v github.com/sensepost/gowitness@latest
!pip install uro requests

print("✅ Instalação concluída. Próximo: Recon básico.")


44 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
make is already the newest version (4.3-4.1build1).
make set to manually installed.
git is already the newest version (1:2.34.1-1ubuntu1.15).
jq is already the newest version (1.6-2.1ubuntu3.1).
unzip is already the newest version (6.0-26ubuntu3.2).
The following additional packages will be installed:
  golang-1.18 golang-1.18-doc golang-1.18-go golang-1.18-src golang-doc
  golang-go golang-src python3-pkg-resources python3-setuptools python3-wheel
Suggested packages:
  bzr | brz mercurial subversion python-setuptools-doc
The following NEW packages will be installed:
  golang golang-1.18 golang-1.18-doc golang-1.18-go golang-1.18-src golang-doc
  golang-go golang-src python3-pip python3-setuptools python3-wheel
The following packag

In [ ]:
# @title 🔍 Fase 1 — Enumeração de Subdomínios
# @markdown Substitua `target.com` pelo domínio desejado.
dominio = "target.com"  # @param {type:"string"}

!subfinder -d {dominio} -all -silent -o subs.txt
!curl -s "https://crt.sh/?q=%25.{dominio}&output=json" | jq -r '.[].name_value' | sed 's/\*\.//g' | sort -u >> subs.txt
!cat subs.txt | anew > subs_unique.txt
!dnsx -l subs_unique.txt -a -resp-only -silent | awk -F, '{print $1}' > resolved.txt
print("✅ Subdomínios resolvidos em resolved.txt")


In [ ]:
# @title 🌐 Fase 2 — HTTP Probe e Screenshot
!httpx -l resolved.txt -silent -threads 200 -status-code -title -tech-detect -follow-redirects -o alive.txt
!cat alive.txt | cut -d' ' -f1 > hosts.txt
!gowitness file -f hosts.txt --threads 10 --timeout 10
print("✅ Hosts vivos listados e screenshots salvos.")


In [ ]:
# @title 🕸️ Fase 3 — Crawling & JS Discovery
!katana -list hosts.txt -d 5 -silent -em js,json -o js_json.txt
!cat js_json.txt | grep -Ei '\.js$' | anew js_files.txt
print("✅ JS e JSON coletados.")


In [ ]:
# @title 🔑 Fase 4 — Segredos e Endpoints
!git clone https://github.com/GerbenJavado/LinkFinder
!git clone https://github.com/m4ll0k/SecretFinder
!xargs -a js_files.txt -P5 -I@ python3 LinkFinder/linkfinder.py -i @ -o cli >> endpoints.txt
!xargs -a js_files.txt -P5 -I@ python3 SecretFinder/SecretFinder.py -i @ -o cli >> secrets.txt
print("✅ Endpoints e possíveis segredos extraídos.")


In [ ]:
# @title ⚔️ Fase 5 — Vulnerability Checks
!cat hosts.txt | naabu -ports 80,443,8080,8443 -silent -rate 10000 -o ports.txt
!nuclei -l hosts.txt -severity low,medium,high,critical -c 50 -o nuclei_scan.txt
!dalfox pipe --input-file js_files.txt -o xss_dalfox.txt
print("✅ Scans básicos de Nuclei e Dalfox concluídos.")


In [ ]:
# @title 🧩 Fase 6 — Coleta e Resumo
!echo "### HOSTS" && cat hosts.txt | wc -l
!echo "### JS" && cat js_files.txt | wc -l
!echo "### ENDPOINTS" && cat endpoints.txt | wc -l
!echo "### SECRETS" && cat secrets.txt | wc -l
!echo "### PORTS" && cat ports.txt | wc -l
!echo "### NUCLEI FINDINGS" && grep -cE "CRITICAL|HIGH" nuclei_scan.txt
print("📊 Resumo final gerado.")
